# Model Inference
This script converts YOLO-format object detection results into a GIS-Friendly GeoJSON file using corresponding World Files (.wld) for georeferencing
Steps:
Reads each YOLO .txt label file from the detection folder
Uses associated .wld files to georeference the bounding boxes
Converts normalised YOLO coordinates into real-world geographic coordinates
Exports the results as a GeoJSON Feature Collection
Output: A single .geojson file with polygon features and class metadata. Useful for visualising detections in QGIS, ArcGIS or other GIS platforms.
 
## Satellite Image Processing Pipeline
 
This notebook processes one or more large high-resolution satellite images through the following automated steps:
 
1. **Tiling**  

   Large GeoTIFF images are split into smaller 512x512 tiles to preserve spatial resolution and reduce GPU memory usage.  

   This allows high-resolution features (like tanks or structures) to remain sharp without downscaling.
 
2. **Conversion to PNG + World Files (.wld)**  

   Each tile is converted to PNG format after contrast enhancement (percentile stretching).  

   A `.wld` file is also created for each tile to retain geospatial referencing.
 
3. **Object Detection using YOLOv8**  

   A pretrained YOLOv8 model is used to detect features (e.g., tanks) in each PNG tile.  

   The model outputs annotated images, `.txt` labels, and confidence scores.
 
4. **GeoJSON Export**  

   YOLO label files are converted to GIS-ready GeoJSON format.  

   Bounding boxes are georeferenced using the corresponding `.tif` metadata.  

   The final `.geojson` file contains polygons with class names and confidence scores — ready for use in QGIS, ArcGIS, or other GIS tools.
 
---
 
### Input

- One or more `.tif` satellite images placed in the input folder
 
### Output

- `.tif` tiles, `.png` images, `.wld` files, YOLO detections, and a final `detections.geojson`
 
---

 

In [ ]:
# Satellite Image Processing Pipeline for Multiple Images
# Description: This script processes all large satellite GeoTIFF images in a folder. It performs:
# 1. Tiling into smaller 512x512 GeoTIFFs
# 2. PNG conversion with contrast stretch + world files
# 3. YOLOv8 inference on the PNG tiles
# 4. Conversion of YOLO results to a unified GeoJSON


#Importing required libraries
import os                          # For folder and file path operations
import json                        # For writing GeoJSON outputs
import cv2                         # For saving images as PNG
import numpy as np                 # For array operations and contrast stretching
import rasterio                    # For reading/writing geospatial raster data
from rasterio.windows import Window  # For tiling large rasters
from pyproj import Transformer     # For converting coordinates from one CRS to another
from ultralytics import YOLO       # For running YOLOv8 object detection
 

# User Configuration

# Folder containing large .tif images
INPUT_FOLDER = "datasets/satellite/"  
# Size of each tile in pixels
TILE_SIZE = 512                            
# Path to trained YOLOv8 model weight
MODEL_PATH = "runs/detect/train/model_weight/best.pt"                
#List of classes the model can detect
CLASS_NAMES = [
    'closed_roof_tank',
    'external_floating_roof_tank',
    'narrow_closed_roof_tank',
    'sedimentation_tank',
    'spherical_tank',
    'undefined_object',
    'water_tower',
    'slurry_tank'
]
 
# Output Directories

#All intermediate files and results willbe stored under the processing directory
PROCESSING_DIR = os.path.join(INPUT_FOLDER, "processing")
TILE_DIR = PROCESSING_DIR                                # GeoTIFF tiles
PNG_DIR = os.path.join(PROCESSING_DIR, "png")            #PNG tiles
WLD_DIR = os.path.join(PROCESSING_DIR, "wld")            #.wld files 
RESULT_DIR = os.path.join(PNG_DIR, "results")             #YOLO results (annotated images)
LABELS_DIR = os.path.join(RESULT_DIR, "labels")          #YOLO generated .txt files 
# Final GeoJSON output path
GEOJSON_OUTPUT = os.path.join(PROCESSING_DIR, "detections.geojson")
 

# Create output directories if not present
os.makedirs(TILE_DIR, exist_ok=True)
os.makedirs(PNG_DIR, exist_ok=True)
os.makedirs(WLD_DIR, exist_ok=True)
os.makedirs(RESULT_DIR, exist_ok=True)
 
# STEP 1: TILE EACH LARGE IMAGE

#Larger satellite images are split into smaller 512x512 tiles
# Done to reduce memory usage and preseve resolution for small features
def tile_images():
    print("\n Step 1: Tiling images...")
    #Loop through all files in the input folder
    for file in os.listdir(INPUT_FOLDER):
        #Process only .tif images
        if file.lower().endswith(".tif"):
            input_path = os.path.join(INPUT_FOLDER, file)
            #Open the large GeoTIFF using Rasterio
            with rasterio.open(input_path) as src:
                tile_num = 1        #Counter to name each tile
                
                #Loop over width and height of the image in steps of TILE_SIZE
                for i in range(0, src.width, TILE_SIZE):
                    for j in range(0, src.height, TILE_SIZE):
                        #Create a window(tile) starting from (i,j)
                        window = Window(i, j, TILE_SIZE, TILE_SIZE)
                        #Copy the image metadata and update it for the tile
                        profile = src.profile.copy()
                        profile.update({
                            "width": TILE_SIZE,
                            "height": TILE_SIZE,
                            "transform": src.window_transform(window)
                        })
                        #Generate a filename for the tile
                        tile_name = f"{os.path.splitext(file)[0]}_tile_{tile_num}.tif"
                        tile_path = os.path.join(TILE_DIR, tile_name)
                        #Write the tile to a new GeoTIFF file
                        with rasterio.open(tile_path, "w", **profile) as dst:
                            dst.write(src.read(window=window)) #Copy pixel data from window
                        tile_num += 1 #Increment tile number
    print("Tiling complete!")
 
# STEP 2: CONVERT TILES TO PNG + WLD

#Preparing the tile GeoTIFF for YOLO input
#Applies contrast enhancement (percentile stretch) for better visualisation
#Converts tile to .png format
#Saves a corresponding .wld file to retain geospatial location

#Applies percentile stretch using specified percentiles
#Enhances image contrast without saturating extermes
def percentile_stretch(img, lower=5, upper=99):
    img = img.astype(np.float32)
    for i in range(img.shape[-1]):         #For each color channel
        low, high = np.percentile(img[..., i], (lower, upper))
        img[..., i] = np.clip((img[..., i] - low) / (high - low) * 255, 0, 255)
    return img.astype(np.uint8)             #Returns contrast enhanced 8-bit image
 
def convert_tiles_to_png():
    print("\n Step 2: Converting tiles to PNG...")
    for file in os.listdir(TILE_DIR):
        if file.endswith(".tif"):
            with rasterio.open(os.path.join(TILE_DIR, file)) as src:
                img = src.read([3, 2, 1])  # Assume BGR order, rearranged
                img = np.moveaxis(img, 0, -1)
                #Apply contrast enhancement
                img = percentile_stretch(img)
                #Save as .png
                png_path = os.path.join(PNG_DIR, file.replace(".tif", ".png"))
                cv2.imwrite(png_path, img, [cv2.IMWRITE_PNG_COMPRESSION, 3])

                # Write corresponding world file

                transform = src.transform
                wld_path = os.path.join(WLD_DIR, file.replace(".tif", ".wld"))
                with open(wld_path, "w") as wld:
                    wld.write(f"{transform.a}\n{transform.b}\n{transform.d}\n{transform.e}\n{transform.xoff}\n{transform.yoff}\n")

    print("PNG conversion and .wld export complete.")
 
# STEP 3: YOLO INFERENCE

# Uses the trained YOLOv8 model to detect objects in PNG tiles
# Saves:
# Annotated images
# YOLO .txt label files
# Confidence scores

def run_yolo():
    print("\n Step 3: Running YOLO inference...")
    model = YOLO(MODEL_PATH)
    model.predict(
        source=PNG_DIR,
        imgsz=(TILE_SIZE, TILE_SIZE),
        save=True,
        save_txt=True,
        save_conf=True,
        conf=0.5,
        iou=0.5,
        project=PNG_DIR,
        name="results",
        exist_ok= True,
        line_width=1,
        classes=[7],
        device=0  # GPU ID
    )

    print("YOLO detection complete.")
 
# STEP 4: TXT TO GEOJSON

# Converts all YOLO .txt detection files into a single GeoJSON file

# Steps:
# For each .tif tile: match its corresponding YOLO .txt label file
# Use raster transform + CRS to convert pixel coords to map coords
# Reproject to EPSG:4326 (WGS84)
# Save as GeoJSON polygons with metadata

# Output: detections.geojson (ready for GIS visualization)
 
def txt_to_geojson():
    print("\n Step 4: Converting YOLO results to GeoJSON...")
    geojson = {"type": "FeatureCollection", "features": []}
    for tif_file in os.listdir(TILE_DIR):
        if tif_file.endswith(".tif"):
            base = os.path.splitext(tif_file)[0]
            tif_path = os.path.join(TILE_DIR, tif_file)
            txt_path = os.path.join(LABELS_DIR, base + ".txt")
            #Skip if there are no detections for this file
            if not os.path.exists(txt_path):
                continue

            with rasterio.open(tif_path) as src:
                transform = src.transform
                crs = src.crs
                width, height = src.width, src.height
                #Set up projection from image to ESPG:4326
                transformer = Transformer.from_crs(crs, "EPSG:4326", always_xy=True)
                with open(txt_path, "r") as f:

                    for line in f:
                        parts = list(map(float, line.strip().split()))
                        if len(parts) < 5:
                            continue

                        class_id = int(parts[0])
                        xc, yc, w, h = parts[1:5]
                        conf = parts[5] if len(parts) == 6 else None

                        #Convert YOLO box(normalised) to map coordinates
                        x1 = (xc - w / 2) * width
                        y1 = (yc - h / 2) * height
                        x2 = (xc + w / 2) * width
                        y2 = (yc + h / 2) * height

                        coords = []
                        for px, py in [(x1,y1), (x2,y1), (x2,y2), (x1,y2), (x1,y1)]:
                            mx, my = rasterio.transform.xy(transform, py, px, offset='center')
                            lon, lat = transformer.transform(mx, my)
                            coords.append([lon, lat])
                            
                        #Save as a polygon feature    
                        geojson['features'].append({
                            "type": "Feature",
                            "geometry": {"type": "Polygon", "coordinates": [coords]},
                            "properties": {
                                "class": class_id,
                                "class_name": CLASS_NAMES[class_id] if class_id < len(CLASS_NAMES) else "unknown",
                                "confidence": conf,
                                "source_image": tif_file
                            }
                        })
#Save final GeoJSON file
    with open(GEOJSON_OUTPUT, "w") as f:
        json.dump(geojson, f, indent=2)
    print(f"GeoJSON saved to: {GEOJSON_OUTPUT}")
 
# Function Call

if __name__ == "__main__":
    tile_images()
    convert_tiles_to_png()
    run_yolo()
    txt_to_geojson()
    print("\n All processing steps completed successfully!")

## Alternative Use Case: PNG + JSON Metadata Input
 
Some satellite image providers supply high-resolution **RGB images in PNG format**, accompanied by metadata in **STAC-style JSON** files.
 
In this case: 
- The PNG image has **already undergone contrast enhancement**, so **no further stretching or normalization is required**.
- The JSON metadata typically contains the bounding box (`bbox`), projection information (`proj:epsg`), and optionally the ground sampling distance (`gsd`).
- Each PNG is processed directly using its pixel resolution and geospatial extent to create `.wld` files for spatial referencing.

**This path bypasses the GeoTIFF reading and contrast stretching steps** used in the TIFF + GeoTIFF metadata pipeline.
 
This makes it ideal for:
- Web-optimized satellite imagery
- Image previews where GeoTIFFs are unavailable
- Lightweight deployments where visual quality is pre-processed

### Overview:

This script automates object detection on satellite PNG images with associated JSON metadata. It tiles each image, performs YOLOv8 inference, georeferences detections, and exports a GIS-compatible GeoJSON file.

### Input Assumptions:

- Each .png image must have a matching JSON metadata file in the same folder
- The metadata file must follow the naming convention: <basename>_metadata.json

Example:
```
       image_01.png → image_01_metadata.json

```
Metadata Assumptions:



- Metadata JSON is expected to follow the STAC (SpatioTemporal Asset Catalog) 1.0.0 structure

- Specifically, the following fields must be present:
    - "bbox": [minX, minY, maxX, maxY] in projected coordinate system
    - "properties.proj:epsg": EPSG code for coordinate reference system
    - (Optional) "properties.gsd": Ground Sampling Distance (meters per pixel)

- If "gsd" is missing, the pipeline assumes a default of 0.3 meters/pixel.
> This format is commonly used by some satellite imagery providers.
For non-STAC or custom formats, this script may require modification.

### Output Structure:

For each input image, a folder is created:
```
   processing_<image_basename>/

     ├── png/                → 512x512 tiles as PNG images
     ├── wld/                → .wld world files for each tile
     ├── png/results/        → YOLOv8 predictions (annotated images)
     ├── png/results/labels/ → YOLO-format .txt label files
     └── detections.geojson  → Final GeoJSON with georeferenced bounding boxes
```

### Output CRS:

 - Final GeoJSON is reprojected to EPSG:4326 (WGS84)

 Usage:
 - Place all PNG images and metadata JSONs into INPUT_FOLDER
 - Run this script to generate detections.geojson for each image

In [ ]:
# Satellite Image Processing Pipeline for Multiple PNG + JSON Metadata Images

# Description:
# This script processes all PNG satellite images in a folder, where each image has an accompanying JSON metadata file.
# It performs:
# 1. Tiling each PNG image into 512x512 tiles
# 2. Generating .wld (world) files for each tile using bounding box and resolution from JSON
# 3. Running YOLOv8 inference on each tile
# 4. Converting YOLO detections to a GeoJSON file per image

import os
import json
import cv2
import numpy as np
from pyproj import Transformer
from ultralytics import YOLO

# User Configuration
INPUT_FOLDER = "datasets/satellite/"  # Folder containing PNG + JSON pairs
TILE_SIZE = 512                            # Size of each tile
MODEL_PATH = "runs/detect/train/model_weight/best.pt"  # YOLOv8 model weights

# List of classes that the model can detect
CLASS_NAMES = [
    'closed_roof_tank', 'external_floating_roof_tank', 'narrow_closed_roof_tank',
    'sedimentation_tank', 'spherical_tank', 'undefined_object', 'water_tower', 'slurry_tank'
]

# STEP 1: Extract EPSG Code from JSON Metadata

def extract_epsg_from_json(json_path):
    with open(json_path) as f:
        metadata = json.load(f)
    return metadata["properties"].get("proj:epsg", "EPSG:32632")  # Default fallback

# STEP 2: Tile PNG and Generate .wld Files

def tile_png_with_wld(png_path, json_path, processing_dir, pixel_size):
    PNG_DIR = os.path.join(processing_dir, "png")
    WLD_DIR = os.path.join(processing_dir, "wld")
    os.makedirs(PNG_DIR, exist_ok=True)
    os.makedirs(WLD_DIR, exist_ok=True)

    img = cv2.imread(png_path)
    h, w = img.shape[:2]

    with open(json_path) as f:
        metadata = json.load(f)
    bbox = metadata["bbox"]  # [minX, minY, maxX, maxY] in projection units
    min_x, min_y, max_x, max_y = bbox
    origin_x = min_x
    origin_y = max_y

    tile_id = 1
    for y in range(0, h, TILE_SIZE):
        for x in range(0, w, TILE_SIZE):
            tile = img[y:y+TILE_SIZE, x:x+TILE_SIZE]
            tile_name = f"tile_{tile_id}.png"
            tile_path = os.path.join(PNG_DIR, tile_name)
            cv2.imwrite(tile_path, tile)

            # Calculate real-world top-left coordinate of this tile
            tile_origin_x = origin_x + x * pixel_size
            tile_origin_y = origin_y - y * pixel_size

            # Write the .wld file for this tile
            wld_path = os.path.join(WLD_DIR, tile_name.replace(".png", ".wld"))
            with open(wld_path, "w") as wf:
                wf.write(f"{pixel_size}\n0.0\n0.0\n{-pixel_size}\n{tile_origin_x}\n{tile_origin_y}\n")

            tile_id += 1

    print(f"Tiling and WLD generation complete for {os.path.basename(png_path)}.")
    return PNG_DIR, WLD_DIR, extract_epsg_from_json(json_path)

# STEP 3: Run YOLO Inference on Tiles

def run_yolo(png_dir):
    print("Running YOLO inference...")
    model = YOLO(MODEL_PATH)
    model.predict(
        source=png_dir,
        imgsz=(TILE_SIZE, TILE_SIZE),
        save=True,
        save_txt=True,
        save_conf=True,
        conf=0.5,
        iou=0.5,
        project=png_dir,
        name="results",
        exist_ok=True,
        line_width=1,
        classes=[7],  # Only run inference for 'slurry_tank' class
        device=0
    )
    print("YOLO detection complete.")
    return os.path.join(png_dir, "results", "labels")

# STEP 4: Convert YOLO TXT Detections to GeoJSON

def txt_to_geojson(png_dir, wld_dir, labels_dir, epsg_code, output_geojson):
    print("Converting YOLO results to GeoJSON...")
    geojson = {"type": "FeatureCollection", "features": []}
    transformer = Transformer.from_crs(epsg_code, "EPSG:4326", always_xy=True)

    for file in os.listdir(png_dir):
        if file.startswith("tile") and file.endswith(".png"):
            label_file = os.path.join(labels_dir, file.replace(".png", ".txt"))
            wld_file = os.path.join(wld_dir, file.replace(".png", ".wld"))
            if not os.path.exists(label_file) or not os.path.exists(wld_file):
                continue

            # Read the corresponding .wld file
            with open(wld_file) as wf:
                A = float(wf.readline())  # pixel size X
                D = float(wf.readline())
                B = float(wf.readline())
                E = float(wf.readline())  # pixel size Y
                C = float(wf.readline())  # origin X
                F = float(wf.readline())  # origin Y

            with open(label_file) as lf:
                for line in lf:
                    parts = list(map(float, line.strip().split()))
                    if len(parts) < 5:
                        continue
                    class_id = int(parts[0])
                    xc, yc, w, h = parts[1:5]
                    conf = parts[5] if len(parts) == 6 else None

                    img = cv2.imread(os.path.join(png_dir, file))
                    img_h, img_w = img.shape[:2]

                    # Convert YOLO center-based bbox to pixel corners
                    x1 = (xc - w / 2) * img_w
                    y1 = (yc - h / 2) * img_h
                    x2 = (xc + w / 2) * img_w
                    y2 = (yc + h / 2) * img_h

                    coords = []
                    for px, py in [(x1,y1), (x2,y1), (x2,y2), (x1,y2), (x1,y1)]:
                        mx = A * px + B * py + C
                        my = D * px + E * py + F
                        lon, lat = transformer.transform(mx, my)
                        coords.append([lon, lat])

                    # Append detection to GeoJSON
                    geojson["features"].append({
                        "type": "Feature",
                        "geometry": {"type": "Polygon", "coordinates": [coords]},
                        "properties": {
                            "class": class_id,
                            "class_name": CLASS_NAMES[class_id] if class_id < len(CLASS_NAMES) else "unknown",
                            "confidence": conf,
                            "source_image": file
                        }
                    })

    with open(output_geojson, "w") as f:
        json.dump(geojson, f, indent=2)
    print(f"GeoJSON saved to: {output_geojson}")

# MAIN EXECUTION

if __name__ == "__main__":
    for file in os.listdir(INPUT_FOLDER):
        if file.endswith(".png"):
            base_name = os.path.splitext(file)[0]
            png_path = os.path.join(INPUT_FOLDER, file)
            json_path = os.path.join(INPUT_FOLDER, base_name + "_metadata.json")

            if not os.path.exists(json_path):
                print(f"Skipping {file}: No matching JSON metadata.")
                continue

            processing_dir = os.path.join(INPUT_FOLDER, f"processing_{base_name}")
            pixel_size = 0.3  # Default pixel size if GSD is missing

            # Process image
            png_dir, wld_dir, epsg_code = tile_png_with_wld(png_path, json_path, processing_dir, pixel_size)
            labels_dir = run_yolo(png_dir)
            output_geojson = os.path.join(processing_dir, "detections.geojson")
            txt_to_geojson(png_dir, wld_dir, labels_dir, epsg_code, output_geojson)

    print("\nAll images processed successfully!")